In [1]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False) # ResNet 모델에서 Fully Connected layer 부분을 제거해주는 역할

94765736/94765736 [==============================] - 5s 0us/step


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

base_dir = '/content/drive/MyDrive/apple'

train_dir = os.path.join(base_dir, 'train')
train_fresh_dir = os.path.join(train_dir, 'fresh')
train_rotten_dir = os.path.join(train_dir, 'rotten')

test_apple_dir = os.path.join(base_dir, 'test_apple')
test_apple_fresh_dir = os.path.join(test_apple_dir, 'fresh')
test_apple_rotten_dir = os.path.join(test_apple_dir, 'rotten')

test_banana_dir = os.path.join(base_dir, 'test_banana')
test_banana_fresh_dir = os.path.join(test_banana_dir, 'fresh')
test_banana_rotten_dir = os.path.join(test_banana_dir, 'rotten')

test_orange_dir = os.path.join(base_dir, 'test_orange')
test_orange_fresh_dir = os.path.join(test_orange_dir, 'fresh')
test_orange_rotten_dir = os.path.join(test_orange_dir, 'rotten')

In [4]:
train_fresh_fnames = os.listdir(train_fresh_dir)
train_fresh_fnames.sort()
print(train_fresh_fnames[:10])

train_rotten_fnames = os.listdir(train_rotten_dir)
train_rotten_fnames.sort()
print(train_rotten_fnames[:10])

['Screen Shot 2018-06-08 at 4.59.36 PM.png', 'Screen Shot 2018-06-08 at 4.59.49 PM.png', 'Screen Shot 2018-06-08 at 4.59.57 PM.png', 'Screen Shot 2018-06-08 at 5.00.03 PM.png', 'Screen Shot 2018-06-08 at 5.00.12 PM.png', 'Screen Shot 2018-06-08 at 5.00.18 PM.png', 'Screen Shot 2018-06-08 at 5.00.26 PM.png', 'Screen Shot 2018-06-08 at 5.00.35 PM.png', 'Screen Shot 2018-06-08 at 5.00.43 PM.png', 'Screen Shot 2018-06-08 at 5.00.50 PM.png']
['Screen Shot 2018-06-07 at 2.15.20 PM.png', 'Screen Shot 2018-06-07 at 2.15.50 PM.png', 'Screen Shot 2018-06-07 at 2.16.18 PM.png', 'Screen Shot 2018-06-07 at 2.16.41 PM.png', 'Screen Shot 2018-06-07 at 2.17.15 PM.png', 'Screen Shot 2018-06-07 at 2.17.25 PM.png', 'Screen Shot 2018-06-07 at 2.18.13 PM.png', 'Screen Shot 2018-06-07 at 2.18.57 PM.png', 'Screen Shot 2018-06-07 at 2.19.15 PM.png', 'Screen Shot 2018-06-07 at 2.19.37 PM.png']


In [5]:
print('total training fresh images:', len(os.listdir(train_fresh_dir)))
print('total training rotten images:', len(os.listdir(train_rotten_dir)))
print('total test_apple fresh images:', len(os.listdir(test_apple_fresh_dir)))
print('total test_apple rotten images:', len(os.listdir(test_apple_rotten_dir)))
print('total test_banana fresh images:', len(os.listdir(test_banana_fresh_dir)))
print('total test_banana rotten images:', len(os.listdir(test_banana_rotten_dir)))
print('total test_orange fresh images:', len(os.listdir(test_orange_fresh_dir)))
print('total test_orange rotten images:', len(os.listdir(test_orange_rotten_dir)))

total training fresh images: 1693
total training rotten images: 2342
total test_apple fresh images: 395
total test_apple rotten images: 601
total test_banana fresh images: 381
total test_banana rotten images: 530
total test_orange fresh images: 388
total test_orange rotten images: 403


In [6]:
from tensorflow.keras import layers
img_input = layers.Input(shape=(224*224*3))

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_apple_datagen = ImageDataGenerator(rescale=1./255)
test_banana_datagen = ImageDataGenerator(rescale=1./255)
test_orange_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh','rotten'],
        class_mode="binary")

test_apple_generator = test_apple_datagen.flow_from_directory(
        test_apple_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")


test_banana_generator = test_banana_datagen.flow_from_directory(
        test_banana_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")

test_orange_generator = test_orange_datagen.flow_from_directory(
        test_orange_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")


# 전체데이터 : 배치사이즈 수정 필요
# resnet50 input image size ; 224 x 224 x 3

Found 4035 images belonging to 2 classes.
Found 996 images belonging to 2 classes.
Found 911 images belonging to 2 classes.
Found 791 images belonging to 2 classes.


In [12]:
for layer in resnet.layers:
    layer.trainable = False
# ResNet50 레이어들의 파라미터를 학습하지 않도록 설정합니다.
# 이렇게 설정된 매개 변수는 역전파를 통해 오차 정보가 전파 되더라도 파라미터가 업데이트 되지 않습니다.

In [13]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [14]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
history = model.fit(
      train_generator, 
      epochs=30,
      verbose=1)

Epoch 1/30
16/16 [==============================] - 1928s 119s/step - loss: 0.6798 - accuracy: 0.5767
Epoch 2/30
16/16 [==============================] - 23s 1s/step - loss: 0.6720 - accuracy: 0.5861
Epoch 3/30
16/16 [==============================] - 23s 1s/step - loss: 0.6626 - accuracy: 0.5948
Epoch 4/30
16/16 [==============================] - 22s 1s/step - loss: 0.6582 - accuracy: 0.6055
Epoch 5/30
16/16 [==============================] - 22s 1s/step - loss: 0.6538 - accuracy: 0.6119
Epoch 6/30
16/16 [==============================] - 23s 1s/step - loss: 0.6394 - accuracy: 0.6337
Epoch 7/30
16/16 [==============================] - 23s 1s/step - loss: 0.6289 - accuracy: 0.6478
Epoch 8/30
16/16 [==============================] - 23s 1s/step - loss: 0.6128 - accuracy: 0.6731
Epoch 9/30
16/16 [==============================] - 22s 1s/step - loss: 0.6109 - accuracy: 0.6719
Epoch 10/30
16/16 [==============================] - 22s 1s/step - loss: 0.6092 - accuracy: 0.6659
Epoch 11/30
16/

In [16]:
loss_and_metrics_apple = model.evaluate(test_apple_generator)
print('loss_and_metrics_apple : ' + str(loss_and_metrics_apple))

4/4 [==============================] - 546s 181s/step - loss: 0.4953 - accuracy: 0.7631
loss_and_metrics_apple : [0.49534887075424194, 0.763052225112915]


In [17]:
loss_and_metrics_banana = model.evaluate(test_banana_generator)
print('loss_and_metrics_banana : ' + str(loss_and_metrics_banana))

4/4 [==============================] - 450s 149s/step - loss: 0.8009 - accuracy: 0.6158
loss_and_metrics_banana : [0.8008984327316284, 0.6158068180084229]


In [18]:
loss_and_metrics_orange = model.evaluate(test_orange_generator)
print('loss_and_metrics_orange : ' + str(loss_and_metrics_orange))

4/4 [==============================] - 387s 128s/step - loss: 0.6250 - accuracy: 0.6928
loss_and_metrics_orange : [0.6249644756317139, 0.6927939057350159]


In [19]:
# 모델 저장하기
from keras.models import load_model
model.save('apple_Resnet50_Dense64_128_epoch30_model.h5')

# 2. 모델 불러오기
# from keras.models import load_model
# model = load_model('모델파일명.h5')